In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Preparo los features

In [2]:

installs1 = pd.read_csv('modelos/sc/installs1.csv')
installs2 = pd.read_csv('modelos/sc/installs2.csv')
installs3 = pd.read_csv('modelos/sc/installs3.csv')
installs4 = pd.read_csv('modelos/sc/installs4.csv')
installs5 = pd.read_csv('modelos/sc/installs5.csv')

installsCheck1 = pd.read_csv('modelos/sc/installsCheck1.csv')
installsCheck2 = pd.read_csv('modelos/sc/installsCheck1.csv')
installsCheck3 = pd.read_csv('modelos/sc/installsCheck1.csv')
installsCheck4 = pd.read_csv('modelos/sc/installsCheck1.csv')
installsCheck5 = pd.read_csv('modelos/sc/installsCheck1.csv')


# Target

En base a los target ids armo los features de esos ids para la predicción.

In [3]:
target = pd.read_csv('../datos/target_competencia_ids.csv')

In [4]:
targets_sc = target[target['ref_hash'].str.contains('_sc')]

In [5]:
targets_sc['ref_hash'] = targets_sc['ref_hash'].map(lambda x: str(x)[:-3])
targets_sc['ref_hash'] = targets_sc['ref_hash'].astype(np.int64)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
targets_sc.count()

ref_hash    4037
obj         4037
dtype: int64

In [14]:
X = pd.read_csv('modelos/sc/x.csv')

In [15]:
X.head()

,ref_type,ref_hash,total_apps,%implicit,%attributed,most_freq_lang,most_freq_app,model,ip,events_%wifi,event_apps,distinct_events,total_events
0,1891515180541284343,90072729247980,1,0.0,0.0,3.301378e+18,210,6.832960e+18,7787461885673001416,0.000000,3.0,1.0,3.0
1,1891515180541284343,161514654074162,1,1.0,0.0,6.977049e+18,65,6.684966e+18,7251732924179152213,0.970588,68.0,8.0,68.0
2,1891515180541284343,168103949904656,1,0.0,0.0,3.301378e+18,155,1.816356e+18,2759827436407391547,0.000000,13.0,1.0,13.0
3,1891515180541284343,283297668933729,1,0.0,0.0,3.301378e+18,210,1.805456e+18,3663507817982540667,0.000000,38.0,1.0,38.0
4,1891515180541284343,448610188195811,2,0.5,0.0,3.301378e+18,122,3.324371e+17,2148831140161787882,0.000000,62.0,6.0,62.0


In [24]:
X.count()

ref_type           125407
ref_hash           125407
total_apps         125407
%implicit          125407
%attributed        125407
most_freq_lang     125407
most_freq_app      125407
model              125407
ip                 125407
events_%wifi       125407
event_apps         125407
distinct_events    125407
total_events       125407
dtype: int64

Ahora en _X_ tengo los features necesarios para los ids que quiero predecir.

# Cargo los modelos entrenados en _PromedioPrediccionesSc.ipynb_
## XBoost

In [7]:
# Cargo el modelo del archivo
xg_reg = joblib.load('modelos/sc/xg_reg.pkl')  
  
predictions_sc_xgb = xg_reg.predict(X)

## Bagging

In [8]:
bagging = joblib.load('modelos/sc/bagging.pkl')  
predictions_sc_bagging = bagging.predict(X)

## AdaBoost

In [9]:
regr = joblib.load('modelos/sc/regr.pkl') 
predictions_sc_adaboost = regr.predict(X)

## LGBM

In [10]:
gbm = joblib.load('modelos/sc/gbm.pkl')
predictions_sc_lgbm = gbm.predict(X)

## Random Forests

In [11]:
rf = joblib.load('modelos/sc/rf.pkl')
predictions_sc_rf = rf.predict(X)

# Promedio las predicciones y actualizo el target

In [12]:
predictions_sc = np.add(np.add(np.add(np.add(predictions_sc_xgb,predictions_sc_bagging), predictions_sc_adaboost), predictions_sc_lgbm),predictions_sc_rf)
predictions_sc = predictions_sc / 5


ValueError: Length of values does not match length of index

In [16]:
targets_sc.head()

,ref_hash,obj
0,1000169251625791246,0
2,1000395625957344683,0
4,1003027494996471685,0
6,1006670001679961544,0
8,1007573308966476713,0


In [17]:
targets_sc.count()

ref_hash    4037
obj         4037
dtype: int64

In [22]:
len(predictions_sc)

125407

In [23]:
predictions_sc

array([128429.28232055, 117399.71595924, 117505.20330268, ...,
       120793.36010363, 115987.95552827, 108505.68156104])

In [ ]:
targets_sc['obj'] = predictions_sc
target.update(targets_sc)
target

In [ ]:
target = pd.read_csv('../datos/preds-promedios.csv')

In [ ]:
target.count()

## Blending

#### Ventana 1

In [ ]:
X_test, y_test = installsCheck1.iloc[:,:-1], installsCheck1.iloc[:,-1]

In [ ]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)

In [ ]:
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm

In [ ]:
regr.fit(X_val, y_val)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### Ventana 2

In [ ]:
X_test, y_test = installsCheck2.iloc[:,:-1], installsCheck2.iloc[:,-1]
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)

In [ ]:
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### Ventana 3

In [ ]:
X_test, y_test = installsCheck3.iloc[:,:-1], installsCheck3.iloc[:,-1]
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### Ventana 4

In [ ]:
X_test, y_test = installsCheck4.iloc[:,:-1], installsCheck4.iloc[:,-1]
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### Ventana 5

In [ ]:
X_test, y_test = installsCheck5.iloc[:,:-1], installsCheck5.iloc[:,-1]
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [ ]:
preds_xgb = xg_reg.predict(X_val)
preds_rf = rf.predict(X_val)
preds_bag = bagging.predict(X_val)
preds_lgbm = gbm.predict(X_val)
test_preds_xgb = xg_reg.predict(X_test)
test_preds_rf = rf.predict(X_test)
test_preds_bag = bagging.predict(X_test)
test_preds_lgbm = gbm.predict(X_test)
X_val['pred_xgb'] = preds_xgb
X_val['pred_rf'] = preds_rf
X_val['pred_bag'] = preds_bag
X_val['pred_lgbm'] = preds_lgbm
X_test['pred_xgb'] = test_preds_xgb
X_test['pred_rf'] = test_preds_rf
X_test['pred_bag'] = test_preds_bag
X_test['pred_lgbm'] = test_preds_lgbm

preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
test = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 'total_events']]

In [ ]:
preds_xgb = xg_reg.predict(test)
preds_rf = rf.predict(test)
preds_bag = bagging.predict(test)
preds_lgbm = gbm.predict(test)

In [ ]:
test['preds_xgb'] = preds_xgb
test['preds_rf'] = preds_rf
test['preds_bag'] = preds_bag
test['preds_lgbm'] = preds_lgbm

In [ ]:
test.to_csv('modelos/sc/test_sc.csv', index=False)

In [ ]:
predictions_sc = regr.predict(test)
predictions_sc

In [ ]:
predictions_sc.size

In [ ]:
pd.DataFrame(predictions_sc).to_csv("../output/predicciones_sc.csv", header=None, index=False)